In [ ]:
import tensorflow as tf
import random
from PIL import Image
from utils import visualization_utils as vis_util
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
PATH_TO_CKPT =  'trainedModels/ssd_mobilenet_RoadDamageDetector.pb' 

PATH_TO_LABELS = 'trainedModels/crack_label_map.pbtxt'

NUM_CLASSES = 8

In [ ]:
detection_graph = tf.Graph()

with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
from utils import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [5]:
import numpy as np

def load_image_into_numpy_array(image):
    """
    Загружаем изображение и преобразуем, к нампи-массиву
    """
    
    im_width, im_height = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [7]:
import os
directory = '/media/jacob/OS/workspace/road_damage_detector-master/asdasdasdasd/'
list_files = os.listdir(directory)

In [12]:
test_img = ['/media/jacob/OS/workspace/road_damage_detector-master/asdasdasdasd/Muroran_20170911100821.jpg']

In [ ]:
with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
    
        # объявляем вхдные и выходные тензоры для графа
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Каждый бокс - часть изображения, где объекты уже обнаружены 
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # тензоры класса скоров
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for num , image_path in enumerate(test_img):
            # Читаем картинку
            image = Image.open(image_path) #directory + list_files
            
            # оборачиваем в нампи массив
            image_np = load_image_into_numpy_array(image)
            print (image_np.shape)
            # расширяем : [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            """
            Порог срабатывания по боксам оставить около 0.3 по скору
            
            """
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
#             print('boxes: {}, scores: {}, class: {}, num: {}'.format(boxes,scores,classes,num))
                        
            print('boxes: {}, scores: {}, class: {}, num: {}'.format(boxes.shape,scores.shape,classes.shape,num.shape))
                                                                     
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                min_score_thresh=0.3,
                use_normalized_coordinates=True,
                line_thickness=8)
            plt.figure(figsize=(12,8))
            plt.imshow(image_np)